In [1]:
from transformers import AdamW 

2023-03-01 19:44:47.387759: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-01 19:44:47.567417: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-03-01 19:44:48.176680: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-01 19:44:48.176746: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

In [2]:
from datasets import concatenate_datasets
from datasets import DatasetDict
from transformers import AutoTokenizer
import numpy as np
from datasets import Dataset

from torchmetrics import R2Score
from torch.nn import MSELoss
from torch.nn import L1Loss
from torch.nn import CrossEntropyLoss


from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm, trange

import torch
import torch.nn as nn
import torch.nn.functional as F
import tokenizers

import pandas as pd
import typing

/media/nvme3n1/proj_scisen/venv/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /media/nvme3n1/proj_scisen/venv/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN3c106detail19maybe_wrap_dim_slowEllb
  warn(f"Failed to load image Python extension: {e}")


In [3]:
from transformers import get_linear_schedule_with_warmup

In [4]:
LAMBDA=0.2
BATCH_SIZE=4
EPOCHS=10
LEARNING_RATE = 0.0005
DROPOUT = 0.3
WEIGHT_DECAY = 0.05
tokenizer_name = "bert-base-uncased"

In [5]:
SCI_PATHS : list() = ['/media/nvme3n1/proj_scisen/datasets/SciSen-as.txt',
                        '/media/nvme3n1/proj_scisen/datasets/SciSen-a.txt',
                        '/media/nvme3n1/proj_scisen/datasets/SciSen-b.txt',
                        '/media/nvme3n1/proj_scisen/datasets/SciSen-c.txt']
NON_SCI_PATHS: list() = ['/media/nvme3n1/proj_scisen/datasets/ScifiSen1.txt',
                          '/media/nvme3n1/proj_scisen/datasets/RedditSen.txt',
                          "/media/nvme3n1/proj_scisen/datasets/ukraineTweets.txt",
                          '/media/nvme3n1/proj_scisen/datasets/ScifiSen2.txt']

In [6]:
sci_sentences : list() = []
non_sci_sentences : list() = []

eval_dict= {0 : None,
           1 : None,
           2 : None,
           3 : None}
rank_sentences : list() = []

test_size = 500
for paths in range(0, len(SCI_PATHS)):
    with open(SCI_PATHS[paths]) as f:
        for line in f:
            rank_sentences.append(line)
    sci_sentences = sci_sentences + (rank_sentences[0:int(len(rank_sentences)*0.9)])
    eval_dict[paths] = rank_sentences[int(len(rank_sentences)*0.9):len(rank_sentences)]
    rank_sentences=[]



index = 0
for path in NON_SCI_PATHS:
    with open(path) as f:
        for line in f:
            non_sci_sentences.append(line) 
            

In [7]:
len(sci_sentences)

3154189

In [8]:
len(non_sci_sentences)

487443

In [9]:
input_list : list() = []

for i in tqdm(range(0,int(len(sci_sentences)))):
    input_list.append({**{'text': sci_sentences[i].rstrip("\n")}, 'labels':0.9})

100%|████████████████████████████████████████████████████████████████████| 3154189/3154189 [00:02<00:00, 1414541.29it/s]


In [10]:
for i in tqdm(range(0,int(len(non_sci_sentences)))):
    input_list.append({**{'text': non_sci_sentences[i].rstrip("\n")}, 'labels':0.1})

100%|██████████████████████████████████████████████████████████████████████| 487443/487443 [00:00<00:00, 1469624.38it/s]


In [11]:
df_input = pd.DataFrame(input_list)

In [12]:
type(0.1)

float

In [13]:
df_input=df_input.mask(df_input == '')
df_input = df_input[~df_input['text'].isnull()]

In [14]:
train_size = 0.77
valid_size = 0.22

#split into train, test:
trainvalid_df=df_input.sample(frac= train_size + valid_size ,random_state=200)
test_dataset=df_input.drop(trainvalid_df.index).reset_index(drop=True)
trainvalid_df = trainvalid_df.reset_index(drop=True)


#split into train, validation:
train_dataset=trainvalid_df.sample(frac= train_size/(train_size + valid_size) ,random_state=200)
valid_dataset=trainvalid_df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(df_input.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("VALIDATION Dataset: {}".format(valid_dataset.shape))
print("FULL TEST Dataset: {}".format(test_dataset.shape))


FULL Dataset: (3641632, 2)
TRAIN Dataset: (2804057, 2)
VALIDATION Dataset: (801159, 2)
FULL TEST Dataset: (36416, 2)


In [15]:
N = int(len(train_dataset)) + int(len(valid_dataset))

In [16]:
train_dataset

,text,labels
0,We increase the number of residual blocks to 6...,0.9
1,We propose an end-to-end trained architecture ...,0.9
2,This led to the model generating both that the...,0.9
3,This clearly indicates that a single approach ...,0.9
4,And it is interesting to verify that <equation...,0.9
...,...,...
2804052,The query result from <equation> from the netw...,0.9
2804053,"Thus, we carefully design FAME as an augmentat...",0.9
2804054,Do you live on in ignorance (and potentially b...,0.1
2804055,"To train our contextual knowledge ranker, we s...",0.9


In [17]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

In [18]:
N

3605216

In [19]:
train_data = np.array(train_dataset['text'].values.tolist(), dtype=object)
valid_data = np.array(valid_dataset['text'].values.tolist(), dtype=object)
raw_documents = np.append(train_data, valid_data)

train_labels = np.array(train_dataset['labels'].values.tolist(), dtype=object)
test_labels_dict = dict()
valid_labels = np.array(valid_dataset['labels'].values.tolist(), dtype=object)
labels = np.append(train_labels.data, valid_labels.data)


print(f"Encoding documents without max_length")
enc_docs = [tokenizer.encode(raw_doc, truncation=True, max_length=512) for raw_doc in raw_documents]

print("Encoding labels...")
#label2index = {label: idx for idx, label in enumerate(labels_in_order)}

enc_labels = []
idx = 0
labels_in_order = ["score"]
label2index = {label: idx for idx, label in enumerate(labels_in_order)}

train_mask, valid_mask = torch.zeros(N, dtype=torch.bool), torch.zeros(N, dtype=torch.bool)

for array in train_labels:
    label_names = np.array(array)
    array_ids = np.empty(1)
    array_ids.fill(0)
    train_mask[idx] = True
    idx += 1
    #for label_name in label_names:
    #  for label, index in label2index.items():
    #        if label_name == label:
    #            array_ids[index] = 1
    enc_labels.append(array)
        
for array in valid_labels:
    label_names = np.array(array)
    array_ids = np.empty(1)
    array_ids.fill(0)
    valid_mask[idx] = True
    idx += 1
    #for label_name in label_names:
    #    for label, index in label2index.items():
    #        if label_name == label:
    #            array_ids[index] = 1
    enc_labels.append(array)




Encoding documents without max_length


KeyboardInterrupt: 

In [ ]:
enc_docs_arr, enc_labels_arr = np.array(enc_docs, dtype='object'), np.array(enc_labels)

train_data = list(zip(enc_docs_arr[train_mask], enc_labels_arr[train_mask]))
valid_data = list(zip(enc_docs_arr[valid_mask], enc_labels_arr[valid_mask]))

In [20]:
def collate_for_mlp(list_of_samples):
    """ Collate function that creates batches of flat docs tensor and offsets """
    offset = 0
    flat_docs, offsets, labels = [], [], []
    for doc, label in list_of_samples:
        if isinstance(doc, tokenizers.Encoding):
            doc = doc.ids
        offsets.append(offset)
        flat_docs.extend(doc)
        labels.append(label)
        offset += len(doc)

    return torch.tensor(np.array(flat_docs)), torch.tensor(np.array(offsets)), torch.tensor(np.array(labels))



In [21]:
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"


In [22]:
class MLP(nn.Module):
    """Simple MLP"""

    def __init__(self, vocab_size, num_classes,
                 num_hidden_layers=1,
                 hidden_size=1024, hidden_act='relu',
                 dropout=0.5, idf=None, mode='mean',
                 pretrained_embedding=None, freeze=True,
                 embedding_dropout=0.5):
        nn.Module.__init__(self)
        # Treat TF-IDF mode appropriately
        mode = 'sum' if idf is not None else mode
        self.idf = idf

        # Input-to-hidden (efficient via embedding bag)
        if pretrained_embedding is not None:
            # vocabsize is defined by embedding in this case
            self.embed = nn.EmbeddingBag.from_pretrained(pretrained_embedding, freeze=freeze, mode=mode)
            embedding_size = pretrained_embedding.size(1)
            self.embedding_is_pretrained = True
        else:
            assert vocab_size is not None
            self.embed = nn.EmbeddingBag(vocab_size, hidden_size, mode=mode)
            embedding_size = hidden_size
            self.embedding_is_pretrained = False

        self.activation = getattr(F, hidden_act)
        self.embedding_dropout = nn.Dropout(embedding_dropout)
        self.dropout = nn.Dropout(dropout)
        self.layers = nn.ModuleList()

        # Hidden-to-hidden
        for i in range(num_hidden_layers - 1):
            if i == 0:
                self.layers.append(nn.Linear(embedding_size, hidden_size))
            else:
                self.layers.append(nn.Linear(hidden_size, hidden_size))

        # Hidden-to-output
        self.layers.append(nn.Linear(hidden_size, num_classes))
        #self.layers.append(nn.Softmax(hidden_size if self.layers else embedding_size, num_classes))
        #self.layers.append(nn.Softmax())
        # Loss function
        self.loss_function = nn.MSELoss()


    def forward(self, input, offsets, labels=None):
        # Use idf weights if present
        
        
        idf_weights = self.idf[input] if self.idf is not None else None

        h = self.embed(input, offsets, per_sample_weights=idf_weights)
        if self.idf is not None:
            # In the TF-IDF case: renormalize according to l2 norm
            h = h / torch.linalg.norm(h, dim=1, keepdim=True)

        if not self.embedding_is_pretrained:
            # No nonlinearity when embedding is pretrained
            h = self.activation(h)

        h = self.embedding_dropout(h)

        for i, layer in enumerate(self.layers):
            # at least one
            h = layer(h)
            if i != len(self.layers) - 1:
                # No activation/dropout for final layer
                h = self.activation(h)
                h = self.dropout(h)

        if labels is not None: 
            labels = labels.to(torch.float64)
            h = h.to(torch.float64)
            loss = self.loss_function(h.squeeze(), labels)            
            return loss, h
        return h


In [23]:
import logging
logging.basicConfig(format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt='%m/%d/%Y %H:%M:%S',
                    level=logging.INFO)

logger = logging.getLogger(__name__)  



os.environ["TOKENIZERS_PARALLELISM"] = "false"  

import wandb
WANDB = True  
model_id = "WideMLP_2s_32_0.1" #for filenames
wandb_id = "WideMLP-FULL"
wandb_project = "wideclassifier"

if WANDB:
    wandb.init(project=wandb_project, name = wandb_id, config={"epochs": EPOCHS, "dropout": DROPOUT, "batch_size": BATCH_SIZE, "learning_rate": LEARNING_RATE, "lambdas": LAMBDA, "trainingdata":"full", })

wandb: Currently logged in as: justinmuecke (scise). Use `wandb login --relogin` to force relogin


In [24]:
import torch
torch.cuda.is_available()
torch.cuda.device_count()

1

In [25]:
GRADIENT_ACCUMULATION_STEPS = 1
LOGGING_STEPS = 50  
torch_device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

print("Initializing MLP")

vocab_size = tokenizer.vocab_size
model = MLP(vocab_size, 1)

model.to(torch_device)
if WANDB:
    wandb.watch(model, log_freq=LOGGING_STEPS)

Initializing MLP


In [26]:

def get_collate_for_transformer(pad_token_id):
    """ Closure to include padding in collate function """
    def _collate_for_transformer(examples):
        docs, labels = list(zip(*examples))
        input_ids = torch.tensor(pad(docs, with_token=pad_token_id))
        attention_mask = torch.ones_like(input_ids, dtype=torch.long)
        attention_mask[input_ids == pad_token_id] = 0
        labels = torch.tensor(labels)
        token_type_ids = torch.zeros_like(input_ids)
        return input_ids, attention_mask, token_type_ids, labels
    return _collate_for_transformer

In [ ]:
len(train_data)

In [ ]:
#collate_fn = get_collate_for_transformer(tokenizer.pad_token_id)
collate_fn = collate_for_mlp
train_loader = torch.utils.data.DataLoader(train_data,
                                               collate_fn=collate_fn,
                                               shuffle=True,
                                               batch_size=BATCH_SIZE,
                                               num_workers=0,
                                               pin_memory=('cuda' in torch_device))

valid_loader = torch.utils.data.DataLoader(valid_data,
                                               collate_fn=collate_fn,
                                               shuffle=True,
                                               batch_size=BATCH_SIZE,
                                               num_workers=0,
                                               pin_memory=('cuda' in torch_device))

# len(train_loader) no of batches
t_total = len(train_loader) // GRADIENT_ACCUMULATION_STEPS * EPOCHS

optimizer = AdamW(model.parameters(), lr=LEARNING_RATE, eps=1e-8, weight_decay=WEIGHT_DECAY)
# scheduler = WarmupLinearSchedule(optimizer, warmup_steps=args.warmup_steps, t_total=t_total)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0,
                                                num_training_steps=t_total)
writer = SummaryWriter()

# Train!
logger.info("***** Running training *****")
logger.info("  Num examples = %d", len(train_data))
logger.info("  Num Epochs = %d", EPOCHS)
logger.info("  Batch size  = %d", BATCH_SIZE)
logger.info("  Total train batch size (w. accumulation) = %d",
            BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS)
logger.info("  Gradient Accumulation steps = %d", GRADIENT_ACCUMULATION_STEPS)
logger.info("  Total optimization steps = %d", t_total)

torch.set_default_dtype(torch.float64)
global_step = 0
loss_vals = []
tr_loss, logging_loss, nb_val_steps, vl_loss = 0.0, 0.0, 0.0, 0.0
model.zero_grad()
#train_iterator = trange(EPOCHS, desc="Epoch")
for epoch in range(EPOCHS):
    print("Epoch: ", epoch)
    for step, batch in enumerate(tqdm(train_loader, desc="Iteration")):
        model.train()
        batch = tuple(t.to(torch_device) for t in batch)
        
        outputs = model(batch[0], batch[1], batch[2])
        loss = outputs[0]
        
        if GRADIENT_ACCUMULATION_STEPS > 1:
            loss = loss / GRADIENT_ACCUMULATION_STEPS
        loss.backward()
        tr_loss += loss.item()
        if (step + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
            optimizer.step()
            scheduler.step()
            model.zero_grad()
            global_step += 1
            if WANDB:
                wandb.log({'epoch': epoch,
                            'lr': scheduler.get_last_lr()[0],
                            'loss': loss})

        if LOGGING_STEPS > 0 and global_step % LOGGING_STEPS == 0:
            avg_loss = (tr_loss - logging_loss) / LOGGING_STEPS
            writer.add_scalar('lr', scheduler.get_last_lr()[0], global_step)
            writer.add_scalar('loss', avg_loss, global_step)
            logging_loss = tr_loss

    for step, batch in enumerate(tqdm(valid_loader, desc="Validating")):
        model.eval()
        batch = tuple(t.to(torch_device) for t in batch)
        with torch.no_grad():
            outputs = model(batch[0], batch[1], batch[2])

        nb_val_steps += 1
        loss, logits = outputs[:2]
        vl_loss += loss.mean().item()

    vl_loss /= nb_val_steps
    loss_vals.append(vl_loss)
    if WANDB:
        wandb.log({'val_loss': vl_loss})

writer.close()
#return global_step, tr_loss / global_step

In [ ]:
torch.save(model.state_dict(), "Models/WideMLP")

In [12]:
model

OrderedDict([('embed.weight',
              tensor([[ 1.6026e-04,  7.2114e-04,  1.2834e-03,  ...,  1.1757e-03,
                        1.0334e-03,  1.1002e-08],
                      [-1.9059e-39, -9.9950e-39,  6.0637e-39,  ...,  1.7448e-38,
                        6.2921e-39, -1.4666e-38],
                      [-9.7377e-39,  6.1381e-39,  7.5940e-39,  ..., -2.3575e-39,
                       -7.7914e-39,  2.4283e-39],
                      ...,
                      [ 9.5778e-39,  5.5740e-39,  1.0309e-39,  ..., -1.0942e-39,
                       -1.0446e-39,  1.5163e-38],
                      [-1.6233e-38,  7.6666e-39, -8.2495e-40,  ..., -2.3722e-39,
                        1.0887e-38,  6.5681e-39],
                      [ 6.2092e-39, -4.3323e-40,  1.2024e-38,  ..., -1.0052e-38,
                        1.0064e-38, -1.2710e-38]], device='cuda:0')),
             ('layers.0.weight',
              tensor([[ 0.1001,  0.0735,  0.0661,  ...,  0.0705,  0.1022, -0.4329]],
                   

In [27]:
def evalRanks(input):
    
    enc_in = [tokenizer.encode(sentence, truncation=True, max_length=512) for sentence in input]
    test_labels = np.array([0.9 for i in range(len(input))] , dtype=object)
    enc_labels = []
    
    idx = 0
    N=len(input)
    train_mask, valid_mask = torch.zeros(N, dtype=torch.bool), torch.zeros(N, dtype=torch.bool)

    for array in test_labels:
        label_names = np.array(array)
        array_ids = np.empty(1)
        array_ids.fill(0)
        train_mask[idx] = True
        idx += 1
        enc_labels.append(array)
        
        
    enc_docs_arr, enc_labels_arr = np.array(enc_in, dtype='object'), np.array(enc_labels)
    test_data = list(zip(enc_docs_arr[train_mask], enc_labels_arr[train_mask]))
    
    return test_data



In [28]:
from typing import List

def predictValues(test_data : List) -> List[int]:
    data = collate_for_mlp(test_data)
    tensor = model(data[0].to(torch_device), data[1].to(torch_device))
    return [tensor.detach().cpu().numpy()[entry][0] for entry in range(len(test_data))]

In [29]:
def average(values):
    averageScore = 0.0
    for score in values:
        averageScore = averageScore + score
    averageScore = averageScore / len(values)
    return averageScore

In [30]:
import math
def standDev(values, averageScore):
    sd = 0.0
    for score in values:
        sd = sd + abs(averageScore - score)**2 
    sd = sd/len(values)
    sd = math.sqrt(sd)
    return sd

    

In [37]:
def export(values, avg, sd):
    with open("SentenceEvals/WideMLP-4", "w") as target: 
        target.write("Mean :\n" + str(avg) +"\n")
        target.write("Standard Deviation:\n" + str(sd) + "\n\n")
        target.write("Values:\n" + str(values) + "\n\n\n")


In [ ]:

import matplotlib.pyplot as plt

def visualize(values, rank):
    
    vals = [0.0 for i in range(0,1000)]
    entries : int = 0

    for score in values:
        score = float(score) * 1000 
        score = int(score)
        if score > 999:
            score = 999
        
        vals[score] = vals[score] + 1
        entries = entries + 1

    x_axis = [i for i in range(0,1000)]
    y_axis = vals
    plt.bar(x_axis, y_axis)
    plt.title('Value Distributions')
    plt.xlabel('Score')
    plt.ylabel('Occurance')
    plt.savefig("Plots/WideMLP" + "-" + str(rank) + ".pdf")
    plt.show()

In [32]:
import pickle
class Analysis(object):
    def __init__(self, values, average, sd):
        self.values = values
        self.average = average
        self.sd = sd
        
    def __enter__(self):
        return __reps__(self)

    def __exit__(self, type, value, tb):
        self.stream.close()
        
    def __repr__(self):
        return "avg: " + str(self.average) + "\nStandDev: " + str(self.sd) + "\nValues: " + str(self.values)
    


In [ ]:
for rank in range(len(eval_dict)):
    data = evalRanks(eval_dict[rank])
    values = predictValues(data)
    values = sorted(values)
    averageScore = average(values)
    deviation = standDev(values, averageScore)
    result = Analysis(values, averageScore, deviation) 
    file = open("Pickled_Object/WideMLP-4", "wb")
    pickle.dump(result, file)
    export(values, averageScore, deviation)


# Investigate How Tokens influence Scoring


In [33]:
NON_SCI_PATH = '/media/nvme3n1/proj_scisen/datasets/ScifiSen9.txt'
model.load_state_dict(torch.load("Models/WideMLP"))

<All keys matched successfully>

In [56]:
import random

def insert_token(sentence): 
    x = random.randint(0,99)
    if x % 2 == 0:
        sentence_arr = sentence.split(" ") 
        sentence_arr.insert(random.randint(0, len(sentence_arr)), "<equation>")
        return " ".join(sentence_arr)
    else:
        sentence_arr = sentence.split(" ") 
        sentence_arr.insert(random.randint(0, len(sentence_arr)), "<citation>")
        return " ".join(sentence_arr)       

In [34]:
modified_sentences: list() = []
original_sentences : list() = []

eval_dict= {0 : None}
           #,1 : None}

with open(NON_SCI_PATH) as f:
    for line in f:
        original_sentences.append(line) 
        modified_sentences.append(insert_token(line))
eval_dict[0] = original_sentences
eval_dict[1] = modified_sentences
    


In [38]:
for rank in range(len(eval_dict)):
    data = evalRanks(eval_dict[rank])
    values = predictValues(data)
    values = sorted(values)
    averageScore = average(values)
    deviation = standDev(values, averageScore)
    result = Analysis(values, averageScore, deviation) 
    file = open("Pickled_Object/WideMLP-4", "wb")
    pickle.dump(result, file)
    export(values, averageScore, deviation)